# Inicjalizacja

In [1]:
%matplotlib inline

from evaluator import evaluate, calc_impact, create_functions_in_db, calc_prs_bugginess, overwrite_bugginess_function
from definitions import Repository
import db
import smells
import metrics
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


#plots size in inches
plt.rcParams["figure.figsize"] = (10,5)

In [2]:
from configuration import ProjectConfiguration

config = ProjectConfiguration()

db.prepare(config.connstr)

dbsession = db.get_session()
create_functions_in_db(dbsession)

# Ewaluacja

In [3]:
repo = config.project

if dbsession.query(Repository).filter(Repository.full_name == repo).first() is None:
    raise LookupError("Repository does not exist")

## Udział smelly prs wśród badanych prs (reprodukcja)

In [4]:
smells_evaluations = [
    evaluate(repo, smells.lack_of_review),
    evaluate(repo, smells.missing_description),
    evaluate(repo, smells.large_changesets),
    evaluate(repo, smells.sleeping_reviews),
    evaluate(repo, smells.review_buddies),
    evaluate(repo, smells.ping_pong),
    evaluate(repo, smells.union,
         [smells.lack_of_review,
          smells.missing_description,
          smells.large_changesets,
          smells.sleeping_reviews,
          smells.review_buddies,
          smells.ping_pong]),
    evaluate(repo, smells.intersection,
         [smells.lack_of_review,
          smells.missing_description,
          smells.large_changesets,
          smells.sleeping_reviews,
          smells.review_buddies,
          smells.ping_pong])
]

In [ ]:
# display results as a text
for evaluation in smells_evaluations:
    print(evaluation)

In [ ]:
# display results as a plot
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
xs = list(map(lambda e: e.evaluator_name.split("\n")[0], smells_evaluations))
ys = list(map(lambda e: e.percentage*100, smells_evaluations))
ax.barh(xs, 100, color="forestgreen", label="non-smelly prs")
ax.barh(xs, ys, color="red", label="smelly prs")
for i, v in enumerate(ys):
    ax.text(101, i + 0.1, f"{int(v)}%", color="red")
    ax.text(105, i + 0.1, f"{100-int(v)}%", color="green")
ax.invert_yaxis()
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.1))
plt.title(f"Percentage of smelly and non-smelly prs in {repo}")
plt.margins(x=0.1)
plt.show()

## Wpływ smelli na prawdopodobieństwo wprowadzenia błędu

In [7]:
# evaluate for smells impact
impact_evaluations = list(map(lambda pair: (pair[0], calc_impact(session = dbsession,
                                                                 repo = dbsession.query(Repository).filter(Repository.full_name == repo).first(),
                                                                 evaluator = pair[1],
                                                                 evaluator_args=None)), [
    ("Lack of code review", smells.lack_of_review),
    ("Missing PR description", smells.missing_description),
    ("Large changeset", smells.large_changesets),
    ("Sleeping reviews", smells.sleeping_reviews),
    ("Review Buddies", smells.review_buddies),
    ("Ping-pong reviews", smells.ping_pong)
]))

In [ ]:
# display results as text
print(
    f"Percentage of pulls where at least one file was changed next by bug solving PR:")
print(f"{''.ljust(30)}OK    \t SMELLY\t IMPACT")

for (name, res) in impact_evaluations:
    print(f"{name.ljust(30)}{(res[0] * 100):.2f}%\t {(res[1] * 100):.2f}%\t {'+' if res[1]>res[0] else ''}{((res[1]-res[0]) * 100):.2f}%")

In [ ]:
# display results as a plot
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
xs = list(map(lambda e: e[0], impact_evaluations))
for_ok = list(map(lambda e: e[1][0]*100, impact_evaluations))
for_smelly = list(map(lambda e: e[1][1]*100, impact_evaluations))
ind = np.arange(len(xs))
width = 0.4

ax.barh(ind, for_ok, width, color="forestgreen", label="Percentage for non-smelly")
for i, v in enumerate(for_ok):
    ax.text((v + 1 if v<95 else 95) if v==v else 1, i + 0.1, f"{int(v)}%" if v==v else "no prs",  color=("green" if v<95 else "black"))
ax.barh(ind + width, for_smelly, width, color="red", label="Percentage for smelly")
for i, v in enumerate(for_smelly):
    ax.text((v + 1 if v<95 else 95) if v==v else 1, i + width + 0.1, f"{int(v)}%" if v==v else "no prs", color=("red" if v<95 else "black"))
ax.set(yticks=ind + width/2, yticklabels=xs, ylim=[2*width-1, len(xs)], xlim=[0, 100])
ax.invert_yaxis()
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.1))
plt.title(f"Percentage of pulls where at least one file was changed next by bug solving PR for {repo}")
plt.show()

## Metryki

In [ ]:
from evaluator import get_considered_prs

repo_obj = dbsession.query(Repository).filter(Repository.full_name == repo).first()
prs = get_considered_prs(session = dbsession, repo = repo_obj)

Funkcja zliczająca metrykę bugginess może zostać częściowo nadpisana, domyślnie jest ona następująca:
$
\left\{\begin{array}{ll}
    f(0)=0 &\\
    f(n)=f(x_{n-1})+\frac{n^{-2}}{\text{number of files in original PR}} &\text{gdy n-ty PR zmieniający plik naprawia defektu}\\
    f(n)=f(n-1) &\text{gdy n-ty PR zmieniający plik nie naprawia defektu}
  \end{array}\right.
$

Można nadpisać środkowe równanie.

Domyślnie $n\in[0; 4]\cap\mathbb{N}$, jednak głębokość przeszukiwania również może zostać nadpisana.

In [11]:
# bugginess function can be overwritten, below current expression
# overwrite_bugginess_function(dbsession, "res + pow(i,-2)/number_of_files")

In [12]:
metrics_evaluations = [
    metrics.review_window_metric(prs, repo_obj),
    metrics.review_window_per_line_metric(prs, repo_obj),
    calc_prs_bugginess(repo_obj, prs)
    # can be changed for depth other than 4 (below no limit)
    # calc_prs_bugginess(repo_obj, prs, depth=None)
]

# create dataframe
df = pd.DataFrame()
for m in metrics_evaluations:
    df[m.metric_name] = m.to_list(dbsession)

In [13]:
# fig = plt.figure()
# ax = fig.add_axes([0,0,1,1])
# ax.boxplot(list(map(lambda m: m.to_list(dbsession), metrics_evaluations)),
#     showfliers=False, notch=True, vert=False)
# plt.show()

In [14]:
# calculate correlationn
corr = df.corr()

In [ ]:
# display correlation matrix
print(corr)

In [ ]:
# display correlation heatmap
hm = sns.heatmap(corr, annot = True)
hm.set(title = "Correlation matrix of reviews metrics and bugginess metrics\n")
plt.show()

In [ ]:
# display plots of bugginess against each metrics
upper_limit = df.quantile(0.95)

fig, axs = plt.subplots(len(df.columns)-1)

index = 0
for column_name in df.columns[:-1]:
    col = getattr(df, column_name)
    xs = df[(col < upper_limit[column_name]) & (df.bugginess < upper_limit["bugginess"])][column_name]
    ys = df[(col < upper_limit[column_name]) & (df.bugginess < upper_limit["bugginess"])]["bugginess"]
    axs[index].plot(xs, ys, "ro")
    axs[index].set_xlabel(f"{column_name}\nommited {len(df)-len(xs)} of {len(df)} prs")
    axs[index].set_ylabel("bugginess")
    index += 1

plt.subplots_adjust(top=1, bottom=0, hspace=0.25, wspace=1)
fig.set_figheight((len(df.columns)-1)*fig.get_figheight())
plt.show()